# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

Ramen restaurants have become so popular in the United States now it seems that there is one on every corner, not only in major cities but also in smaller cities. Starting a Ramen restaurant can be a great business opportunity, but you need to distinguish yourself from others to enjoy long-term success.

My Client wants to open his business in Manhattan area, so I only focus on that borough during my analysis. The objective is to locate and recommend to the management which neighborhood of Newyork city will be best choice to start a restaurant. The Management also expects to understand the rationale of the recommendations made.


## Data<a name="data"></a>
Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to ramen restaurants in the neighborhood, if any
To find the best location for our ramen restaurant, we will use the following sources of information:

Newyork has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572


From Foursquare Venues Categories - https://developer.foursquare.com/docs/resources/categories ramen restaurant category Id - 55a59bace4b013909087cb24

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Download and Explore Dataset
Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


### tranform the data into a pandas dataframe

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Getting all data New York City with borough Manhattan, and showing them in the map.


In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Foursquare venues

In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'GEUIZ43UZYPYQQF3XD131M0YCA530S22YOOZKNEER4BWORJB'
CLIENT_SECRET = 'UK2GKTOCADIX41RDZJG01KRSWMW3WH2ORKAIVZC43E3GC1MB'
VERSION = '20191020'

## Methodology <a name="methodology"></a>

In [10]:
#https://developer.foursquare.com/docs/resources/categories
#Ramen = 55a59bace4b013909087cb24
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_Ramen = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='55a59bace4b013909087cb24')
newyork_venues_Ramen.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Tofu Tofu,40.717605,-73.995304,Korean Restaurant
1,Chinatown,40.715618,-73.994279,KURO-OBI at CANAL STREET MARKET,40.719008,-74.000843,Ramen Restaurant
2,Chinatown,40.715618,-73.994279,K&S BBQ Ramen,40.718395,-73.991517,BBQ Joint
3,Chinatown,40.715618,-73.994279,RESOBOX East Village,40.724755,-73.987620,Ramen Restaurant
4,Chinatown,40.715618,-73.994279,Yoyoka,40.722710,-73.988962,Ramen Restaurant


In [11]:
newyork_venues_Ramen.shape

(446, 7)

#### Getting all ramen restaurant in New York City (446), and showing them in the map.

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_Ramen = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_Ramen, 'red', map_newyork_Ramen)

map_newyork_Ramen

In [14]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [15]:
manhattan_grouped = newyork_venues_Ramen.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_ramen['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,3,3,3,3,3,3
Carnegie Hill,4,4,4,4,4,4
Central Harlem,2,2,2,2,2,2
Chelsea,7,7,7,7,7,7
Chinatown,18,18,18,18,18,18
Civic Center,11,11,11,11,11,11
Clinton,24,24,24,24,24,24
East Village,18,18,18,18,18,18
Financial District,3,3,3,3,3,3


### Analyzing each neighborhood by venue category.

In [16]:
 #one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_Ramen[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_Ramen['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Burger Joint,Japanese Restaurant,Korean Restaurant,Poke Place,Ramen Restaurant,Street Food Gathering,Sushi Restaurant
0,Chinatown,0,0,0,0,1,0,0,0,0
1,Chinatown,0,0,0,0,0,0,1,0,0
2,Chinatown,0,1,0,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,1,0,0
4,Chinatown,0,0,0,0,0,0,1,0,0


In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,BBQ Joint,Burger Joint,Japanese Restaurant,Korean Restaurant,Poke Place,Ramen Restaurant,Street Food Gathering,Sushi Restaurant
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
1,Carnegie Hill,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.500000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.857143,0.000000,0.142857
4,Chinatown,0.055556,0.055556,0.000000,0.000000,0.055556,0.000000,0.833333,0.000000,0.000000
5,Civic Center,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.909091,0.000000,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
7,East Village,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.944444,0.000000,0.000000
8,Financial District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
9,Flatiron,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.846154,0.000000,0.076923


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
1,Carnegie Hill,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
2,Central Harlem,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
3,Chelsea,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
4,Chinatown,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant


### Clustering by K-means

In [20]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 0, 3, 3, 2, 3, 2, 0], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,NaN,NaN,NaN,NaN,NaN,NaN
1,Manhattan,Chinatown,40.715618,-73.994279,3.0,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,NaN,NaN,NaN,NaN,NaN,NaN
5,Manhattan,Manhattanville,40.816934,-73.957385,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
7,Manhattan,East Harlem,40.792249,-73.944182,NaN,NaN,NaN,NaN,NaN,NaN
8,Manhattan,Upper East Side,40.775639,-73.960508,1.0,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
9,Manhattan,Yorkville,40.775930,-73.947118,1.0,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering


#### drop the NaN values

In [22]:
manhattan_merged.dropna()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Manhattan,Chinatown,40.715618,-73.994279,3.0,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
8,Manhattan,Upper East Side,40.775639,-73.960508,1.0,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
9,Manhattan,Yorkville,40.775930,-73.947118,1.0,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
10,Manhattan,Lenox Hill,40.768113,-73.958860,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
11,Manhattan,Roosevelt Island,40.762160,-73.949168,2.0,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
12,Manhattan,Upper West Side,40.787658,-73.977059,4.0,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place
13,Manhattan,Lincoln Square,40.773529,-73.985338,4.0,Ramen Restaurant,Sushi Restaurant,Japanese Restaurant,Street Food Gathering,Poke Place


In [23]:

manhattan_merged["Cluster Labels"] = manhattan_merged["Cluster Labels"].fillna(0).astype("int")

In [24]:
manhattan_merged.shape

(40, 10)

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0

In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,NaN,NaN,NaN,NaN,NaN
3,Inwood,NaN,NaN,NaN,NaN,NaN
4,Hamilton Heights,NaN,NaN,NaN,NaN,NaN
7,East Harlem,NaN,NaN,NaN,NaN,NaN
17,Chelsea,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
18,Greenwich Village,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Korean Restaurant,BBQ Joint
21,Tribeca,Ramen Restaurant,Street Food Gathering,Korean Restaurant,Sushi Restaurant,Poke Place
23,Soho,Ramen Restaurant,Street Food Gathering,Korean Restaurant,BBQ Joint,Sushi Restaurant
24,West Village,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
25,Manhattan Valley,NaN,NaN,NaN,NaN,NaN


#### Cluster 1

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Upper East Side,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
9,Yorkville,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering
30,Carnegie Hill,Ramen Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering


#### Cluster 2

In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Washington Heights,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
5,Manhattanville,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
6,Central Harlem,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
10,Lenox Hill,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
11,Roosevelt Island,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
14,Clinton,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
26,Morningside Heights,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
28,Battery Park City,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
29,Financial District,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant
37,Stuyvesant Town,Ramen Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,Korean Restaurant


#### Cluster 3

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,OK for the ramen restaurant
1,Chinatown,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant,N
19,East Village,Ramen Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,N
20,Lower East Side,Ramen Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant,Street Food Gathering,N
22,Little Italy,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant,N
27,Gramercy,Ramen Restaurant,Asian Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,N
31,Noho,Ramen Restaurant,Korean Restaurant,BBQ Joint,Asian Restaurant,Sushi Restaurant,N
32,Civic Center,Ramen Restaurant,Korean Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place,N
33,Midtown South,Ramen Restaurant,Burger Joint,Asian Restaurant,Sushi Restaurant,Street Food Gathering,N


#### Cluster 4

In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Upper West Side,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place
13,Lincoln Square,Ramen Restaurant,Sushi Restaurant,Japanese Restaurant,Street Food Gathering,Poke Place
15,Midtown,Ramen Restaurant,Poke Place,Japanese Restaurant,Burger Joint,Asian Restaurant
16,Murray Hill,Ramen Restaurant,Poke Place,Japanese Restaurant,Burger Joint,Asian Restaurant
34,Sutton Place,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Street Food Gathering,Poke Place
35,Turtle Bay,Ramen Restaurant,Poke Place,Japanese Restaurant,Asian Restaurant,Sushi Restaurant
36,Tudor City,Ramen Restaurant,Poke Place,Japanese Restaurant,Asian Restaurant,Sushi Restaurant


## Results and Discussion <a name="results"></a>

Here we  K-means to clustering data area with less number of ramen restaurants
Based on dataframe analysis above Cluster 1 (Upper East Side,Yorkville,cargie hill ) areas are the best places to open a ramen restaurant.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify  Manhattan area in New York City in order to aid Owner in narrowing down the search for optimal location for a new Ramen restaurant.Here K-means clustering is used on the data to narrow the best possible locations to open the restaurants in Manhattan area of Newyork city 


Final decission on optimal restaurant location will be made by owners based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.